# 추가 맥주 목록 수집

# Ratebeer 수집단계

In [12]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [13]:
# 추가맥주 리스트
added_beer_list = pd.read_csv('추가맥주리스트.csv', encoding='utf-8')
added_beer_list.columns = ['인덱스', '검색이름']
added_beer_list.head(2)

,인덱스,검색이름
0,15,"5,0 Original"
1,18,Gaffel


In [14]:
added_beer_list

,인덱스,검색이름
0,15,"5,0 Original"
1,18,Gaffel
2,20,DAB
3,22,Diebels Alt
4,24,Leikeim
...,...,...
651,3,Slavutich
652,4,Kasztelan
653,5,Brok
654,6,Bieckert


In [38]:
# 수집할 맥주 종류 기입
added_beers = added_beer_list['검색이름'].to_list()
len(added_beers)

656

In [16]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

In [17]:
chromedriver = 'chromedriver.exe'
url = 'https://www.ratebeer.com/search?tab=beer'

In [18]:
driver = webdriver.Chrome(chromedriver)
# driver.get(url)
driver.set_window_size(900, 900)
time.sleep(1)

## 실제 구현 함수

In [27]:
def crawl(driver, beer, data, k):
    
    # Check 변수
    check = 0
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])
    # url
    url = 'https://www.ratebeer.com/search?tab=beer'
    
    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)
    
    # 맥주 검색
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()    

    # 상품 이름 수집"
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text
    
    error_cnt = 0
    
    while 1:
        try :
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string = driver.find_element_by_class_name('MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 로직
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]
            
            print('성공... while문을 탈출합니다.')
            break
        except :
            print('오류 발생.. 다시 시작합니다.')
            
            error_cnt += 1
            
            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return
                
    
    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)
        
    # Score breakdown 클릭
    time.sleep(3)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
#     element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
#     element = driver.find_element_by_css_selector('.MuiButtonBase-root.MuiChip-root.RatingsContainer___StyledChip2-iqbJXI.xGfYx.mr-4.MuiChip-outlined.MuiChip-sizeSmall.MuiChip-clickable')
    time.sleep(3)
    driver.execute_script("arguments[0].click();", element)
    
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')

        # 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        beer_info = driver.find_elements_by_css_selector('.px-4.fj-s.f-wrap')
        
        tmp = []
        
        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])

        # div, span, title 태그 후 속성은 class 외에도 사용 가능
        try :
            element = driver.find_element_by_xpath('//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    if num != len(data):
        data = data[:num]
    
    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))
    
#     result = pd.merge(data, beer_list, on='검색이름', how='left')
    data.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    data.to_excel("beer_n_"+str(k)+".xlsx")
    
    driver.quit()
    
    return data

In [28]:
len(added_beers)

656

In [44]:
# 542번 Argo -> Argo larger beer
# 478번 Craft -> Craft Athens Larger
added_beer_list['검색이름'].iloc[542] = 'Argo larger beer'
added_beer_list['검색이름'].iloc[478] = 'Craft Athens Larger'

C:\Users\Ghyeon\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [49]:
# 수집할 맥주 종류 기입
added_beers = added_beer_list['검색이름'].to_list()
len(added_beers)

656

In [45]:
print(added_beer_list.iloc[478])

인덱스                     543
검색이름    Craft Athens Larger
Name: 478, dtype: object


In [46]:
added_beer_list.to_csv('추가맥주리스트.csv', encoding='utf-8')

In [53]:
for k in range(326, 351):
    result = crawl(driver, added_beers[k], data, k)

url_open... Kastel 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
리뷰수 :  40 수집된 리뷰수 :  40
url_open... Nektar 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
리뷰수 :  260 수집된 리뷰수 :  260
url_open... CISK 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
22 번째 페이지입니다.
23 번째 페이지입니다.
24 번째 페이지입니다.
리뷰수 :  350 수집된 리뷰수 :  350
url_open... HOPLEAF 맥주 데이터를 수집합니다..
성공... while문을 탈출합니다.
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8